In [ ]:
import re
import pandas as pd
# Sample data
data = pd.read_csv('/content/drive/MyDrive/Method/EnglishMethod/Java_Train.csv')

In [ ]:
print(data)

                                  Functional Description  \
0                         get the role of this object      
1                        get the state of this object      
2      called by the browser or applet viewer to info...   
3      gets the accessible context associated with th...   
4      determines this applet ' s context , which all...   
...                                                  ...   
17734  allows an interceptor to query the information...   
17735  this operation is invoked when the orb receive...   
17736  returns the object id identifying the object i...   
17737  returns reference to the poa implementing the ...   
17738  receives requests issued to any corba object i...   

                       Method Name  
0              get accessible role  
1         get accessible state set  
2                          destroy  
3           get accessible context  
4               get applet context  
...                            ...  
17734               

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download required resources
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def to_base_form(token):
    return lemmatizer.lemmatize(token, get_wordnet_pos(token))

In [ ]:
df = pd.DataFrame(data)
# Import defaultdict from the collections module
from collections import defaultdict
import inflect

# Initialize inflect engine
p = inflect.engine()
# Accumulate counts
position_counts = defaultdict(lambda: defaultdict(int))
fd_token_occurrences = defaultdict(int)
# Function to extract tokens
def extract_tokens(text):
    # Convert text to lowercase and split into tokens
    tokens = text.lower().split()
    # Convert plural tokens to singular, if applicable, and verbs to base form
    processed_tokens = []
    for token in tokens:
        singular_token = p.singular_noun(token)
        if singular_token:
            # If a singular form is returned, use it
            token = singular_token
        # Lemmatize the token (noun or verb to base form)
        base_form_token = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        processed_tokens.append(base_form_token)
    return processed_tokens

# Accumulate counts
position_counts = defaultdict(lambda: defaultdict(int))
fd_token_occurrences = defaultdict(int)

for _, row in df.iterrows():
    fd_tokens = extract_tokens(row['Functional Description'])
    mn_tokens = extract_tokens(row['Method Name'])

    for fd_token in fd_tokens:
        fd_token_occurrences[fd_token] += 1
        if fd_token in mn_tokens:
            index = mn_tokens.index(fd_token)
            if index == 0:
                position_counts[fd_token]['prefix'] += 1
            elif index == len(mn_tokens) - 1:
                position_counts[fd_token]['suffix'] += 1
            else:
                position_counts[fd_token]['infix'] += 1

# Calculate conditional probabilities
position_probabilities = defaultdict(dict)
for token, positions in position_counts.items():
    total = fd_token_occurrences[token]
    for position in ['prefix', 'infix', 'suffix']:
        position_probabilities[token][position] = positions[position] / total

# Creating a DataFrame to display results
result_data = {
    'Token': [],
    'Prefix Probability': [],
    'Infix Probability': [],
    'Suffix Probability': []
}

for token, probabilities in position_probabilities.items():
    result_data['Token'].append(token)
    result_data['Prefix Probability'].append(probabilities.get('prefix', 0))
    result_data['Infix Probability'].append(probabilities.get('infix', 0))
    result_data['Suffix Probability'].append(probabilities.get('suffix', 0))

probability_df = pd.DataFrame(result_data)

print(probability_df)

            Token  Prefix Probability  Infix Probability  Suffix Probability
0             get            0.875530           0.001414            0.000000
1            role            0.000000           0.371429            0.314286
2           state            0.012712           0.097458            0.165254
3         destroy            0.875000           0.000000            0.000000
4      accessible            0.000000           0.833333            0.013333
...           ...                 ...                ...                 ...
1668  interceptor            0.000000           0.000000            0.230769
1669        codec            0.250000           0.000000            0.000000
1670       expect            0.000000           0.000000            0.034483
1671         most            0.000000           0.022727            0.000000
1672          poa            0.000000           0.000000            0.500000

[1673 rows x 4 columns]


In [ ]:
# Specify the filename
filename = 'pp.csv'

# Save the DataFrame to a CSV file
probability_df.to_csv(filename, index=False)

In [ ]:
import pandas as pd
import inflect

# Initialize inflect engine
p = inflect.engine()

# Function to extract tokens and convert plurals to singular
def extract_tokens(text):
    # Convert text to lowercase and split into tokens
    tokens = text.lower().split()
    # Convert plural tokens to singular, if applicable, and verbs to base form
    processed_tokens = []
    for token in tokens:
        singular_token = p.singular_noun(token)
        if singular_token:
            # If a singular form is returned, use it
            token = singular_token
        # Lemmatize the token (noun or verb to base form)
        base_form_token = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        processed_tokens.append(base_form_token)
    return processed_tokens
# Your functional description
functional_description = "sets the background color of this object"

# Tokenize the functional description
tokens = extract_tokens(functional_description)

# Variables to keep track of top probabilities and corresponding tokens
top_prefix = ('', 0)  # (token, probability)
top_infix = ('', 0)
top_suffix = ('', 0)

# Find the most probable position for each token
for token in tokens:
    if token in probability_df['Token'].values:
        row = probability_df[probability_df['Token'] == token]
        # Extract probabilities
        prefix_prob = row['Prefix Probability'].values[0] * 100
        infix_prob = row['Infix Probability'].values[0] * 100
        suffix_prob = row['Suffix Probability'].values[0] * 100

        # Update top tokens if current token has higher probability and meets the threshold
        if prefix_prob > top_prefix[1] and prefix_prob >= 50:
            top_prefix = (token, prefix_prob)
        if infix_prob > top_infix[1] and infix_prob >= 50:
            top_infix = (token, infix_prob)
        if suffix_prob > top_suffix[1] and suffix_prob >= 50:
            top_suffix = (token, suffix_prob)

# Format final output with probabilities
output = f"In this description, '{top_prefix[0]}' is most likely to be the prefix with a probability of {top_prefix[1]:.2f}%"
if top_infix[0]:
    output += f", '{top_infix[0]}' the infix with a probability of {top_infix[1]:.2f}%"
if top_suffix[0]:
    output += f", and '{top_suffix[0]}' the suffix with a probability of {top_suffix[1]:.2f}%"
output += " in the method name."

print(output)

In this description, 'set' is most likely to be the prefix with a probability of 60.20%, and 'background' the suffix with a probability of 70.91% in the method name.


In [ ]:
import pandas as pd
import inflect
import re

# Initialize inflect engine for singularization
p = inflect.engine()

df = pd.DataFrame(data)

# Tokenize and preprocess descriptions
def preprocess_and_tokenize(text):
     # Convert text to lowercase and split into tokens
    tokens = text.lower().split()
    # Convert plural tokens to singular, if applicable, and verbs to base form
    processed_tokens = []
    for token in tokens:
        singular_token = p.singular_noun(token)
        if singular_token:
            # If a singular form is returned, use it
            token = singular_token
        # Lemmatize the token (noun or verb to base form)
        base_form_token = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        processed_tokens.append(base_form_token)
    return processed_tokens

# Process each functional description
def find_top_tokens(description):
    tokens = preprocess_and_tokenize(description)
    top_prefix = top_infix = top_suffix = (None, 0)  # Initialize with None and 0 probability

    for token in tokens:
        if token in probability_df['Token'].values:
            token_data = probability_df[probability_df['Token'] == token]
            # Extract probabilities and check if they are the highest found so far above the threshold
            prefix_prob = token_data['Prefix Probability'].values[0]
            infix_prob = token_data['Infix Probability'].values[0]
            suffix_prob = token_data['Suffix Probability'].values[0]

            if prefix_prob >= 0.5 and (top_prefix[1] is None or prefix_prob > top_prefix[1]):
                top_prefix = (token, prefix_prob)
            if infix_prob >= 0.5 and (top_infix[1] is None or infix_prob > top_infix[1]):
                top_infix = (token, infix_prob)
            if suffix_prob >= 0.5 and (top_suffix[1] is None or suffix_prob > top_suffix[1]):
                top_suffix = (token, suffix_prob)

    return (top_prefix[0], top_infix[0], top_suffix[0]), (top_prefix[1], top_infix[1], top_suffix[1])

# Apply the function to each description and unpack results into new columns
results = df['Functional Description'].apply(find_top_tokens)
df['Prefix'], df['Infix'], df['Suffix'] = zip(*results.apply(lambda x: x[0]))
df['Prefix Probability'], df['Infix Probability'], df['Suffix Probability'] = zip(*results.apply(lambda x: x[1]))
# Show updated DataFrame
print(df)


                                  Functional Description  \
0                         get the role of this object      
1                        get the state of this object      
2      called by the browser or applet viewer to info...   
3      gets the accessible context associated with th...   
4      determines this applet ' s context , which all...   
...                                                  ...   
17734  allows an interceptor to query the information...   
17735  this operation is invoked when the orb receive...   
17736  returns the object id identifying the object i...   
17737  returns reference to the poa implementing the ...   
17738  receives requests issued to any corba object i...   

                       Method Name   Prefix       Infix Suffix  \
0              get accessible role      get        None   None   
1         get accessible state set      get        None   None   
2                          destroy  destroy        None   None   
3           get

In [ ]:
# Specify the filename
filename = 'Engdataset.csv'

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)

In [ ]:
df.head()

,Functional Description,Method Name,Prefix,Infix,Suffix,Prefix Probability,Infix Probability,Suffix Probability
0,get the role of this object,get accessible role,get,None,None,0.87553,0.000000,0.0
1,get the state of this object,get accessible state set,get,None,None,0.87553,0.000000,0.0
2,called by the browser or applet viewer to info...,destroy,destroy,None,None,0.87500,0.000000,0.0
3,gets the accessible context associated with th...,get accessible context,get,accessible,None,0.87553,0.833333,0.0
4,"determines this applet ' s context , which all...",get applet context,None,None,None,0.00000,0.000000,0.0


In [ ]:
def find_token_details(dataframe, description):
    # Ensure the DataFrame and description are prepared for matching
    description = description.strip().lower()
    dataframe['Functional Description'] = dataframe['Functional Description'].astype(str).str.strip().str.lower()

    # Find the matching row
    row = dataframe[dataframe['Functional Description'] == description]

    if not row.empty:
        # Extract and format the output
        output = "In this description, "
        details = []

        # Check and format prefix
        if pd.notna(row['Prefix'].values[0]):
            prefix = row['Prefix'].values[0]
            prefix_prob = row['Prefix Probability'].values[0] * 100  # Convert probability to percentage
            details.append(f"'{prefix}' is most likely to be the prefix with a probability of {prefix_prob:.2f}%")

        # Check and format infix
        if pd.notna(row['Infix'].values[0]):
            infix = row['Infix'].values[0]
            infix_prob = row['Infix Probability'].values[0] * 100  # Convert probability to percentage
            details.append(f"'{infix}' the infix with a probability of {infix_prob:.2f}%")

        # Check and format suffix
        if pd.notna(row['Suffix'].values[0]):
            suffix = row['Suffix'].values[0]
            suffix_prob = row['Suffix Probability'].values[0] * 100  # Convert probability to percentage
            details.append(f"and '{suffix}' the suffix with a probability of {suffix_prob:.2f}%")

        output += ", ".join(details) + " in the method name."
        return output
    else:
        return "Description not found in DataFrame"


# Usage example
description = "get the state of this object"
result = find_token_details(df, description)

print(result)

In this description, 'get' is most likely to be the prefix with a probability of 87.55% in the method name.


In [ ]:
pip install transformers torch

In [ ]:
pip install transformers scipy

In [ ]:
import torch
from transformers import BertModel, BertTokenizer
from scipy.spatial.distance import cosine

# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

def get_token_embeddings(text):
    """Generates BERT embeddings for each token in the input text."""
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
    embeddings = output.hidden_states[-1].squeeze(0)  # Get the embeddings from the last layer
    return embeddings, encoded_input['input_ids']

def find_best_description_tokens(method_name, description):
    """Finds the description token with the highest similarity score for each method name token."""
    name_embeddings, name_ids = get_token_embeddings(method_name)
    desc_embeddings, desc_ids = get_token_embeddings(description)

    name_tokens = [tokenizer.decode([token_id]).strip() for token_id in name_ids[0]]
    desc_tokens = [tokenizer.decode([token_id]).strip() for token_id in desc_ids[0]]

    best_matches = {}

    # Iterate over each token in the method name
    for i, (name_embed, name_token) in enumerate(zip(name_embeddings, name_tokens)):
        if name_token in ['[CLS]', '[SEP]']:
            continue
        highest_score = -1
        best_token = None
        # Compare with each token in the description
        for j, (desc_embed, desc_token) in enumerate(zip(desc_embeddings, desc_tokens)):
            if desc_token in ['[CLS]', '[SEP]'] or desc_token.startswith('##'):
                continue
            similarity = 1 - cosine(name_embed, desc_embed)
            if similarity > highest_score:
                highest_score = similarity
                best_token = desc_token
        best_matches[name_token] = (best_token, highest_score)

    return best_matches

In [ ]:
# Example usage
method_name = "clone"
description = "create a copy of this object "
result = find_best_description_tokens(method_name, description)
print(result)
method_name = "check access"
description = "Verify if the current user has the authority to access the requested resource."
result = find_best_description_tokens(method_name, description)
print(result)
method_name = "save settings"
description = "Persist the user's configuration preferences to ensure changes are not lost."
result = find_best_description_tokens(method_name, description)
print(result)


{'clone': ('object', 0.417935848236084)}
{'check': ('verify', 0.6228334903717041), 'access': ('access', 0.5798786282539368)}
{'save': ('user', 0.5641632080078125), 'settings': ('preferences', 0.5901214480400085)}


In [ ]:
df['Pivot'] = df.apply(lambda row: find_best_description_tokens(row['Method Name'], row['Functional Description']), axis=1)
# Specify the filename
filename = 'Processed_dataset.csv'

# Save the DataFrame to a CSV file
df.to_csv(filename, index=False)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Create a function to generate the description for each row
def generate_description(row):
    return (
        f'1) Functional Description: "{row["Functional Description"]}" '
        f'Method Name: "{row["Mname"]}". '
        f'In this example, "{row["Prefix"]}" is most likely to be the prefix with a probability of {row["Prefix Probability"]:.5f}%, '
        f'"{row["Infix"]}" is the Infix with a probability of {row["Infix Probability"]:.5f}%, and '
        f'"{row["Suffix"]}" is the Suffix with a probability of {row["Suffix Probability"]:.5f}% in the method name. '
        f'More on, semantic similarity between description and method token is "{row["Pivot"]}". '
        f'The base LLM predicted the method name "{row["Predicted_Method"]}", which has an edit distance score of {row["Edit_Score"]} '
        f'compared to the ground truth "{row["Mname"]}".'
    )

# Apply the function to each row and create a new column "Example Description"
df['Example Description'] = df.apply(generate_description, axis=1)

# Display the first few rows to verify the changes
print(df[['Functional Description', 'Mname', 'Example Description']].head())

# Save the modified DataFrame to a new CSV file (optional)
output_csv_file_path = 'output_file_with_descriptions.csv'
df.to_csv(output_csv_file_path, index=False)
